## Attempt 1 - Wrong Answer

In [ ]:
class Solution:
    def calculate(self, s: str) -> int:
        
        ## --> did not consider the edge case where it is a negative number that is returned from parenthesis calculation
        
        def inner(s):
            
            # print(s)
            
            if s[0] == "(" and s[-1] == ")":
                s = s[1:-1]
            
            if "+" not in s and "-" not in s and "*" not in s and "/" not in s:
                return int(s)
            
            if "(" in s:
                l = 0
                for idx in range(len(s)):
                    if s[idx] == "(":
                        l = idx
                    if s[idx] == ")":
                        r = idx
                        break
                tmp = str(inner(s[l : r + 1]))
                return inner(s[:l] + tmp + s[r + 1:])
            
            nums, ops = [], []
            last = 0
            for idx, c in enumerate(s):
                if c not in {"+", "-"}:
                    continue
                nums.append(s[last:idx])
                last = idx + 1
                ops.append(c)
            nums.append(s[last:])
            nums = [n if len(n) > 0 else '0' for n in nums]
            nums = [mul_divide(n) for n in nums]
            
            if len(nums) == 1:
                return int(nums[0])
            
            res = None
            if ops[0] == "+":
                res = nums[0] + nums[1]
            else:
                res = nums[0] - nums[1]
            for idx in range(1, len(ops)):
                if ops[idx] == "+":
                    res += nums[idx + 1]
                else:
                    res -= nums[idx + 1]
            return int(res)
            
        def mul_divide(s):
            
            # print(s)
            
            if "*" not in s and "/" not in s:
                return int(s)
            
            nums, ops = [], []
            last = 0
            for idx, c in enumerate(s):
                if c not in {"*", "/"}:
                    continue
                # print(f"last: {last}")
                # print(f"idx: {idx}")
                nums.append(int(s[last:idx]))
                last = idx + 1
                ops.append(c)
            nums.append(int(s[last:]))
            res = None
            if ops[0] == "*":
                res = nums[0] * nums[1]
            else:
                res = nums[0] // nums[1]
            for idx in range(1, len(ops)):
                if ops[idx] == "*":
                    res *= nums[idx + 1]
                else:
                    res /= nums[idx + 1]
            return res
        
        return inner(s)
                    

## Attempt 2 - Accepted

In [ ]:
from functools import partial

class Solution:
    def calculate(self, s: str) -> int:
        
        def div(b, a=None): 
            return int(a / b) ## --> turncate towards zero
        
        def mul(b, a=None):
            return a * b
        
        def ident(a):
            return a
        
        stk = []
        operand = 0
        sign = 1
        func = ident
        res = 0
        
        for c in s:
            if c.isdigit():
                operand = 10 * operand + int(c)
            
            if c == '+':
                res += func(sign * operand)
                sign = 1
                func = ident
                operand = 0
            
            if c == '-':
                res += func(sign * operand)
                sign = -1
                func = ident
                operand = 0
            
            if c == '*':
                v = func(sign * operand)
                sign = 1
                func = partial(mul, a=v)
                operand = 0
            
            if c =='/':
                v = func(sign * operand)
                sign = 1
                func = partial(div, a=v)
                operand = 0
                
            if c == '(': ## with another open of parentheses, it is like restart the calculation
                stk.append((res, sign, func))
                res = 0
                operand = 0
                sign = 1
                func = ident
            
            if c == ')': ## at the end, just record what is in this parentheses in operand, and reinstate previous layer
                res += func(sign * operand)
                operand = res
                res, sign, func = stk.pop()
                
        res += func(sign * operand)
        return res

## Lessons and Thoughts
- The accepted solution is from leetcode discussion. The tricks are
    - truncate towards 0: `int(a/b)`
    - treat operations as suspensions, with the help of `partial`, one only need to record the last operand and sign.
- Not sure how to do it in `C++`, without `partial` and functions as an object.

